# SimCLR implementation #

Implementation following: https://theaisummer.com/simclr/

In [1]:
import os

import torch
import torchvision.models as models
from torchvision.datasets import STL10, EuroSAT
from torch.utils.data import DataLoader
from torch.multiprocessing import cpu_count
from pytorch_lightning.callbacks import GradientAccumulationScheduler, ModelCheckpoint
from pytorch_lightning import Trainer


from ssl_remote_sensing.data.eurosat_dataset import means, stds
from ssl_remote_sensing.pretext_tasks.simclr.utils import reproducibility
from ssl_remote_sensing.pretext_tasks.simclr.training import SimCLRTraining
from ssl_remote_sensing.pretext_tasks.simclr.augmentation import Augment

In [2]:
def get_stl_dataloader(batch_size, transform=None, split="unlabeled"):
    stl10 = STL10("/Users/alexanderlontke/datasets/", split=split, transform=transform, download=True)
    return DataLoader(dataset=stl10, batch_size=batch_size, num_workers=cpu_count())

## Putting it all together ##

In [3]:
# Machine setup
available_gpus = torch.cuda.device_count()
save_model_path = os.path.join(os.getcwd(), "saved_models/")
print("available_gpus:", available_gpus)

# Run setup
filename = "SimCLR_ResNet18_adam"
save_name = filename + '.ckpt'
resume_from_checkpoint = False


# Model Setup
class Hparams:
    def __init__(self):
        self.epochs = 1  # number of training epochs
        self.seed = 1234  # randomness seed
        self.cuda = False  # use nvidia gpu
        self.img_size = 64  #image shape
        self.save = "./saved_models/"  # save checkpoint
        self.gradient_accumulation_steps = 1  # gradient accumulation steps
        self.batch_size = 64
        self.lr = 1e-3
        self.embedding_size = 128  # papers value is 128
        self.temperature = 0.5  # 0.1 or 0.5
        self.weight_decay = 1e-6


train_config = Hparams()
reproducibility(train_config)

model = SimCLRTraining(
    config=train_config,
    model=models.resnet18(weights=None),
    feat_dim=512,
    norm_means=means,
    norm_stds=stds
)

transform = Augment(
    train_config.img_size, norm_means=means, norm_stds=stds
)

dataset = EuroSAT(
    "/Users/alexanderlontke/datasets/",
    transform=transform,
    download=True
)
data_loader = DataLoader(
    dataset=torch.utils.data.Subset(dataset,range(200)),
    batch_size=train_config.batch_size,
    num_workers=cpu_count()
)

# Needed to get simulate a large batch size
accumulator = GradientAccumulationScheduler(scheduling={0: 1})

checkpoint_callback = ModelCheckpoint(
    filename=filename,
    dirpath=save_model_path,
    every_n_epochs=2,
    save_last=True,
    save_top_k=2,
    monitor='Contrastive loss_epoch',
    mode='min'
)

if resume_from_checkpoint:
    trainer = Trainer(
        callbacks=[accumulator, checkpoint_callback],
        gpus=available_gpus,
        max_epochs=train_config.epochs,
        resume_from_checkpoint=train_config.checkpoint_path
    )
else:
    trainer = Trainer(
        callbacks=[accumulator, checkpoint_callback],
        gpus=available_gpus,
        max_epochs=train_config.epochs
    )

trainer.fit(model, data_loader)
trainer.save_checkpoint(save_name)

available_gpus: 0
DIM MLP input: 512


/Users/alexanderlontke/.conda/envs/ssl-remote-sensing/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/alexanderlontke/.conda/envs/ssl-remote-sensing/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /Users/alexanderlontke/Documents/Uni/St. Gallen/HS_22_23/deep_learning/ssl-remote-sensing/notebooks/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type          | Params
----------------------------------------
0 | model | AddProjection | 11.5 M
1 | loss  | Inf

Optimizer Adam, Learning Rate 0.001, Effective batch size 64


Training: 0it [00:00, ?it/s]

mask torch.Size([128, 128])
exp torch.Size([128, 128])
mask torch.Size([128, 128])
exp torch.Size([128, 128])
mask torch.Size([128, 128])
exp torch.Size([128, 128])
mask torch.Size([16, 16])
exp torch.Size([16, 16])


`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
from torchsummary import summary
summary(model, (3,64,64))

In [7]:
from ssl_remote_sensing.pretext_tasks.simclr.loss import InfoNceLoss
(x1, x2), label = next(iter(data_loader))
z1 = model(x1)
z2 = model(x2)
InfoNceLoss(train_config.batch_size).forward(z1, z2)

tensor(-4.9118, grad_fn=<DivBackward0>)